In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade --no-cache-dir groq


In [ ]:
import groq
import requests 
import json


In [ ]:
API_KEY = "<YOUR_API_KEY>"

API_URL = "<YOUR_API_KEY>  # Update to the correct Groq API endpoint


In [ ]:
import os

from groq import Groq

client = Groq(
    api_key="<YOUR_API_KEY>",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="<YOUR_API_KEY>",
)

In [ ]:
def groq_api_request(prompt, max_tokens=1500):
    try:
        response = client.chat.completions.create(
            model="<YOUR_API_KEY>",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
        )
        return response.choices[0].message.content  # Extracts the response text
    except Exception as e:
        print(f" API Error: {e}")
        return None

In [ ]:
raw_text = """
Military Brilliance (1796-1797): Napoleon emerged as a strategic genius during his Italian Campaign, defeating Austrian forces and expanding French influence.
Coup of 18 Brumaire (1799): He overthrew the Directory and established the Consulate, becoming First Consul of France.
Coronation as Emperor (1804): He crowned himself Emperor, symbolizing his supreme authority over France.
Conquering Europe (1805-1812): Victories at Austerlitz, Jena, and Wagram cemented his dominance, reshaping the European political landscape.
Economic and Legal Reforms: The Napoleonic Code modernized laws, while infrastructure and education reforms strengthened France internally.
Fall:

Peninsular War (1808-1814): Guerrilla warfare in Spain, backed by Britain, drained French resources and morale.
Russian Campaign (1812): The disastrous invasion of Russia led to massive troop losses due to harsh winters and scorched-earth tactics.
Defeat at Leipzig (1813): The Battle of Nations saw a coalition of European powers overpower Napoleon’s forces.
First Exile to Elba (1814): He was forced to abdicate and exiled to Elba, but later escaped and briefly reclaimed power.
Waterloo and Final Exile (1815): His defeat at Waterloo by the Duke of Wellington ended his rule, leading to his exile to Saint Helena, where he died in 1821.

"""

In [ ]:
ner_prompt = f"""
You are a professional NLP model. Extract structured entities from raw text using **Named Entity Recognition (NER)**.
Identify:

- **Characters**: Names of people or fictional characters.
- **Locations**: Places, cities, or fantasy settings.
- **Objects**: Important items (e.g., "ancient key", "magic book").
- **Events**: Key moments in the story.

### Example Output:
{{
  "characters": ["Alice", "The Shadow"],
  "locations": ["Grand Library"],
  "objects": ["Rusty Key", "Glowing Book"],
  "events": ["Alice enters the library", "Alice finds the key"]
}}

### Input:
{raw_text}

Return the extracted **JSON output** in the above format.
"""


ner_output = groq_api_request(ner_prompt)
print("Named Entity Recognition (NER) Output:")
print(json.dumps(ner_output, indent=2))

In [ ]:
sentiment_prompt = f"""
You are an expert in **emotional tone analysis**.  
Analyze the input text and **detect sentiment shifts**.

- **Identify tone changes** (e.g., hopeful → tense → fearful).  
- **Classify emotions** (e.g., Joy, Fear, Suspense, Sadness).  
- **Map emotions to specific parts** of the text.

### Example Output:
{{
  "sentiments": [
    {{
      "scene": "Alice Enters the Library",
      "emotion": "Curiosity",
      "text": "Alice cautiously steps inside, gripping an ancient key."
    }},
    {{
      "scene": "The Shadow Appears",
      "emotion": "Fear",
      "text": "A dark figure moves across the bookshelves, making Alice shiver."
    }}
  ]
}}

### Input:
{raw_text}

Return the sentiment analysis **in JSON format**.
"""

sentiment_output = groq_api_request(sentiment_prompt)
print("Sentiment Analysis Output:")
print(json.dumps(sentiment_output, indent=2))

In [ ]:
dependency_prompt = f"""
You are an NLP expert specializing in **dependency parsing**.  
Your goal is to extract structured dependency relationships from the input text.

### Extraction Rules:
- **Identify Subject-Verb-Object relationships** clearly.
- **Detect cause-and-effect connections** in the text.
- **Extract dependencies** influencing actions or decisions.
- **Ensure a structured JSON output** with NO extra explanations.

### Expected JSON Format:
{{
  "dependencies": [
    {{
      "sentence": "...",
      "subject": "...",
      "verb": "...",
      "object": "...",
      "modifier": "...",
      "cause": "...",
      "effect": "..."
    }}
  ]
}}

### Input:
{raw_text}

### Output:
- Return a valid **JSON object ONLY**.  
- No additional text, explanations, or formatting.
"""


dependency_output = groq_api_request(dependency_prompt)
print("\n🟢 Dependency Parsing Output:")
print(json.dumps(dependency_output, indent=2))


In [ ]:
import json

script_prompt = scene_prompt = f"""
You are a professional scriptwriter. Generate a **multi-step screenplay** from the structured data below. Follow this process:

1️⃣ **Set the Scene** → Describe the setting in cinematic detail.  
2️⃣ **Introduce Characters** → Show their emotions and motivations.  
3️⃣ **Develop Action** → Describe key events in a logical order.  
4️⃣ **Ensure Coherence** → Maintain smooth transitions between scenes.  

### Example Scene Breakdown:

[SCENE 1]  
**Location:** Grand Library  
*A dimly lit hall with golden chandeliers. Alice steps inside, clutching an old key.*  

**Alice** *(whispering)*  
"I hope this key still works..."  

*A shadow moves across the wall. Alice gasps, gripping the key tighter.*  

---

### Input Structured Data:
{ner_output}

Return the **script in screenplay format**.
"""


script_output = groq_api_request(script_prompt)

try:
    
    print(" Script Generation Output:")
    print(script_output)
except json.JSONDecodeError:
    print(" Error: Invalid JSON response from API")
    print("Raw API Output:")
    print(script_output)  # Print raw output for debugging

In [ ]:
# import json

# image_prompt = f"""
# You are an AI visual artist. Generate **cinematic image descriptions** from this script.
# - Maintain **consistent colors, lighting, and character design** across all scenes.  
# - Describe **character expressions, posture, and mood** in each frame.  
# - Ensure the **same visual style** throughout.  

# ### Example:

# **Image 1: Alice Enters the Library**  
# "A massive ancient library with golden chandeliers. Warm candlelight flickers across endless bookshelves. A young girl in a dark blue cloak, her brown hair tied back, holds a rusted key with glowing engravings. Her green eyes scan the room nervously."

# ---

# ### Input Script:
# {script_output}

# Return a **structured JSON list** of image prompts.
# """


# image_prompt_output = groq_api_request(image_prompt)

# try:
    
#     print(" prompt  Generation Output:")
#     print(image_prompt_output)
# except json.JSONDecodeError:
#     print(" Error: Invalid JSON response from API")
#     print("Raw API Output:")
#     print(script_output)  # Print raw output for debugging

In [ ]:
# import json

# def generate_scene_clusters(script_output):
#     """
#     Segments a script into structured scene descriptions.

#     Args:
#         script_output (str): The raw script input.

#     Returns:
#         list: A list of scene clusters as JSON objects.
#     """
#     if not script_output.strip():
#         print("❌ Error: Script output is empty!")
#         return []

#     # Scene segmentation LLM prompt
#     segmentation_prompt = f"""
#     You are a professional screenplay analyst. Your task is to break down the given script into **separate cinematic scenes**.
    
#     - Identify logical scene breaks based on **setting changes, major events, or shifts in time/mood**.
#     - Ensure **each scene is a self-contained description** that can be used for visualization.
#     - Keep descriptions **concise but vivid**, capturing the environment, characters, and action.
#     - Return the results in a structured **JSON list**, where each scene has a unique number.

#     ### Example Output:
#     ```json
#     [
#         {{"scene_number": 1, "scene_description": "A thunderstorm rages over an abandoned house. Lightning flashes, revealing a lone figure at the broken doorway."}},
#         {{"scene_number": 2, "scene_description": "A neon-lit alleyway in Tokyo. A detective in a trench coat exhales smoke, watching as a mysterious woman disappears into the shadows."}}
#     ]
#     ```

#     ### Input Script:
#     {script_output}

#     Return only valid JSON.
#     """

#     # Call the LLM API
#     raw_response = groq_api_request(segmentation_prompt)

#     try:
        

        

#         return raw_response
#         print(raw_response)

#     except (json.JSONDecodeError, TypeError, ValueError) as e:
#         print(f"❌ Error: Invalid JSON response from LLM - {e}")
#         print("Raw API Output:", raw_response)
#         return []


In [ ]:
# def generate_image_prompts(scene_clusters):
#     """
#     Generates cinematic image descriptions from scene clusters.

#     Args:
#         scene_clusters (list): A list of segmented scene descriptions.

#     Returns:
#         list: A list of image descriptions (one per scene).
#     """
#     image_prompts = []
    
#     for i, scene in enumerate(scene_clusters):
#         image_prompt = f"""
#         You are an AI visual artist. Generate a **cinematic image description** for the scene below.

#         - Maintain **consistent colors, lighting, and character design** across all scenes.  
#         - Describe **character expressions, posture, and mood** in each frame.  
#         - Ensure the **same visual style** throughout.  

#         ### Scene {i+1}:
#         "{scene}"

#         Return a **single JSON object** with the key `"description"`.
#         """

#         raw_response = groq_api_request(image_prompt)

#         # Validate the response
#         try:
#             script_output_json = json.loads(raw_response)
#             description = script_output_json.get("description", "")
#             if description:
#                 image_prompts.append(description)
#         except (json.JSONDecodeError, TypeError) as e:
#             print(f"❌ Error: Invalid JSON response for scene {i+1} - {e}")

#     return image_prompts

In [ ]:
def generate_scene_clusters(script_output):
    """
    Segments a script into structured scene descriptions.

    Args:
        script_output (str): The raw script input.

    Returns:
        list: A list of scene clusters as JSON objects.
    """
    segmentation_prompt = f"""
    You are a professional screenplay analyst. Break the script into **scene clusters**.
    - Identify logical scene breaks.
    - Keep descriptions vivid but concise.
    - Return a JSON list of scene clusters.

    ### Input Script:
    {script_output}

    Return only valid JSON.
    """
    
    # Call your LLM API
    raw_response = groq_api_request(segmentation_prompt)  

    try:
        print(raw_response)
        print(3)
        scene_clusters = json.loads(raw_response)
        if not isinstance(scene_clusters, list):
            raise ValueError("Invalid JSON format received from LLM.")
        return scene_clusters
    except (json.JSONDecodeError, TypeError, ValueError) as e:
        print(f"❌ Error: Invalid JSON response - {e}")
        return []

In [ ]:
!pip install diffusers transformers accelerate safetensors omegaconf
!pip install controlnet_aux compel


In [ ]:
!pip install torch torchvision torchaudio xformers
!pip install opencv-python numpy


In [ ]:
from diffusers import StableDiffusionPipeline
import torch


In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device: {device}")

In [ ]:
import os
os.environ.pop("HF_TOKEN", None)  # Remove the existing token

from huggingface_hub import login
login("")


In [ ]:
import os
os.environ["HF_TOKEN"] = ""


In [ ]:
import torch
import json
import matplotlib.pyplot as plt
from diffusers import StableDiffusionPipeline
from transformers import CLIPImageProcessor

# Load Stable Diffusion Pipeline
pipeline = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16
).to("cuda")

def generate_scene_clusters(script_output):
    """
    Segments a script into structured scene descriptions.

    Args:
        script_output (str): The raw script input.

    Returns:
        list: A list of scene clusters as JSON objects.
    """
    segmentation_prompt = f"""
    You are a professional screenplay analyst. Break the script into **scene clusters**.
    - Identify logical scene breaks.
    - Keep descriptions vivid but concise.
    - Return a JSON list of scene clusters.

    ### Input Script:
    {script_output}

    ### Output Format
    cluster_id: <id>,
    scene_id: <id>,
    scene_description: <description>,
    cluster_description: <description>

    Return only valid JSON.
    """
    
    # Call your LLM API
    raw_response = groq_api_request(segmentation_prompt)
    try:
        data = raw_response.replace("```json", "").replace("```","")
        scene_clusters = json.loads(data)
        if not isinstance(scene_clusters, list):
             raise ValueError("Invalid JSON format received from LLM.")
        return scene_clusters
        #return raw_response
    except (json.JSONDecodeError, TypeError, ValueError) as e:
        print(f"❌ Error: Invalid JSON response - {e}")
        return []


def generate_image_prompts(scene_clusters):
    """
    Generates cinematic image descriptions from scene clusters.

    Args:
        scene_clusters (list): A list of segmented scene descriptions.

    Returns:
        list: A list of image descriptions (one per scene cluster).
    """
    image_prompts = []
    
    for i, scene in enumerate(scene_clusters):
        image_prompt = f"""
        You are an AI visual artist. Generate a **cinematic image description** for the scene below.

        - Maintain **consistent colors, lighting, and character design** across all scenes.
        - Describe **character expressions, posture, and mood**.
        - Ensure the **same visual style** throughout.

        ### Scene {i+1}:
        "{scene['scene_description']}"

        Return a **single JSON object** with the key `"description"`.
        """

        raw_response = groq_api_request(image_prompt)

        try:
            data = raw_response.replace("```json", "").replace("```","")
            response_json = json.loads(data)
            description = response_json.get("description", "")
            if description:
                image_prompts.append(description)
            
        except (json.JSONDecodeError, TypeError) as e:
            print(f"❌ Error: Invalid JSON response for scene {i+1} - {e}")

    return image_prompts


def generate_images_from_scenes(script_output):
    """
    Processes a script, generates images for scene clusters, and plots them.

    Args:
        script_output (str): The raw script input.

    Returns:
        list: A list of generated images.
    """
    
    # Step 1: Get scene clusters
    scene_clusters = generate_scene_clusters(script_output)

    if not scene_clusters:
        print("❌ No valid scene clusters generated.")
        return []

    # Step 2: Generate image prompts
    image_prompts = generate_image_prompts(scene_clusters)
    print(image_prompts)
    print(1)

    if not image_prompts:
        print("❌ No valid image prompts generated.")
        return []

    # Step 3: Generate images
    generated_images = []
    print(len(generated_images))
    previous_image = None  # For IP Adapter

    fig, axs = plt.subplots(1, len(image_prompts), figsize=(15, 5))
    print(image_prompts)

    for i, prompt in enumerate(image_prompts):
        print(f"🎨 Generating Image {i+1}: {prompt}")

        generator = torch.Generator(device="cpu").manual_seed(0)

        # Generate first image normally
        if previous_image is None:
            image = pipeline(prompt=prompt, height=768, width=768, generator=generator).images[0]
        else:
            # Use previous image as context for the next one
            pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")
            pipeline.set_ip_adapter_scale(0.6)

            image = pipeline(prompt=prompt, ip_adapter_image=previous_image, num_inference_steps=100, generator=generator).images[0]

        # Store image
        generated_images.append(image)
        previous_image = image  # Set for next iteration

        # Plot image
        axs[i].imshow(image)
        axs[i].axis("off")
        axs[i].set_title(f"Scene {i+1}")

    plt.show()
    return generated_images


In [ ]:
from diffusers import StableVideoDiffusionPipeline
import torch
import numpy as np
import cv2
from IPython.display import display, Video
from PIL import Image

# Load model
model_id = "stabilityai/stable-video-diffusion-img2vid-xt"
pipe = StableVideoDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16").to("cuda")
pipe.enable_model_cpu_offload()
torch.cuda.empty_cache()

generator = torch.manual_seed(42)
final_frames = []  # List to store final frames

# Define the list of input images dynamically
generated_images = generate_images_from_scenes(script_output)  # Replace with actual image variables

# Process images dynamically
frame_sequences = []  # Stores all frame sequences dynamically

for img in generated_images:
    frames = pipe(
        img,
        decode_chunk_size=2,
        generator=generator,
        num_frames=len(generated_images)  # Dynamically setting number of frames
    ).frames  # Use `.frames` instead of `.frames[0]` to get all frames

    frame_sequences.append(frames)  # Store the sequence of frames

# Transition frames count
num_transition_frames = 10

# Generate transitions
for i in range(len(frame_sequences) - 1):  
    frames_a = frame_sequences[i]  # Current scene frames
    frames_b = frame_sequences[i + 1]  # Next scene frames

    transition_frames = []

    for alpha in np.linspace(0, 1, num_transition_frames):  
        # Extract individual frames and ensure they are PIL images
        img_a = frames_a[-1].convert("RGB")  # Convert last frame of first sequence to RGB PIL Image
        img_b = frames_b[0].convert("RGB")   # Convert first frame of next sequence to RGB PIL Image

        # Blend the images
        blended_frame = Image.blend(img_a, img_b, alpha)
        transition_frames.append(blended_frame)

    # Add current scene frames and transition frames to final output
    final_frames.extend(frames_a + transition_frames)

# Append last scene frames
final_frames.extend(frame_sequences[-1])

# Save as video

# Save as video
video_path = ""
fps = 7

# Ensure correct shape
height, width = final_frames[0].size  # PIL size gives (width, height)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

for frame in final_frames:
    frame = np.array(frame, dtype=np.uint8)  # Convert to uint8
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Convert for OpenCV
    out.write(frame)

out.release()

# Display the video
display(Video(video_path, embed=True))
